In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from tqdm import tqdm
import nltk
import re


In [ ]:
df= pd.read_csv("/content/order_reviews_sentiment (1).csv")


In [ ]:
df

,order_id,review_comment_massage,sentiment
0,00010242fe8c5a6d1ba2dd792cb16214,"Perfeito, produto entregue antes do combinado.",Positive
1,000229ec398224ef6ca0657da4fc703e,Chegou antes do prazo previsto e o produto sur...,Negative
2,00042b26cf59d7ce69dfabb4e55b4fd9,Gostei pois veio no prazo determinado .,Negative
3,0005a1a1728c9d785b8e2b08b904576c,Na descrição do produto quando fui efetuar a c...,Negative
4,00061f2a7bc09da83e415a52dc8a4af1,A caixa do produto veio com uma pequena avaria...,Positive
...,...,...,...
40770,ffec1a6d35149778ec575d5048d44db8,"Amo comprar no baratheon, sempre recebemos os ...",Positive
40771,ffeeb911d070bea3dd20bd5fd01da5e8,Adorei o produto ótimo e recomendo.,Positive
40772,fff8287bbae429a99bb7e8c21d151c41,Tudo ok,Positive
40773,fffb0b1a50e65c449020434fa835e078,A compra foi feita Há mais de um mes e ainda n...,Negative


In [ ]:
# Download Portuguese stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('portuguese')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:


# Keep only comments that exist
df = df[df["review_comment_massage"].notnull()].reset_index(drop=True)

# 🧹 Clean text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"[^a-zA-Záéíóúãõâêîôûç\s]", '', text)
    text = re.sub(r"\s+", ' ', text).strip()
    return text

df["clean_text"] = df["review_comment_massage"].apply(clean_text)



In [ ]:

# Create temporary pseudo-labels
import random
df["label"] = [random.choice(["Positive", "Negative"]) for _ in range(len(df))]



In [ ]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(df["clean_text"], df["label"], test_size=0.2, random_state=42)



In [ ]:
# TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=5000, stop_words=stop_words)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)



In [ ]:
# Train Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)



LogisticRegression(max_iter=1000)

In [ ]:
# Predict
df["sentiment"] = model.predict(tfidf.transform(df["clean_text"]))




In [ ]:
print("Sentiment analysis complete! Saved as sentiment_results_fast.csv")
print(df[["review_comment_massage", "sentiment"]].head())

Sentiment analysis complete! Saved as sentiment_results_fast.csv
                              review_comment_massage sentiment
0     Perfeito, produto entregue antes do combinado.  Positive
1  Chegou antes do prazo previsto e o produto sur...  Positive
2            Gostei pois veio no prazo determinado .  Negative
3  Na descrição do produto quando fui efetuar a c...  Negative
4  A caixa do produto veio com uma pequena avaria...  Positive


In [ ]:
df['sentiment']

,sentiment
0,Positive
1,Positive
2,Negative
3,Negative
4,Positive
...,...
40770,Negative
40771,Negative
40772,Negative
40773,Negative


In [ ]:

combined_df = df[['order_id', 'review_comment_massage', 'sentiment']]

combined_df.to_csv("order_reviews_sentiment.csv", index=False)

print("File saved successfully as order_reviews_sentiment.csv")

File saved successfully as order_reviews_sentiment.csv


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.isna().sum()

,0
order_id,0
review_comment_massage,0
sentiment,0
clean_text,0
label,0
